<a href="https://colab.research.google.com/github/lsteffenel/RT0902-IntroML/blob/main/15-Chicago_crime_data_on_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chicago crime dataset analysis
---

Ce notebook permet d'appliquer un peu de vos connaissances à la découverte d'un vrai dataset.

Vous allez effectuer :
 * Lecture, transformation et requêtage avec Apache Spark.
 * Parfois, transformer les données en Pandas pour une meilleure visualisation.


---

## Quelques Import



Import de Pandas et Matplotlib

In [ ]:
## standard imports
import pandas as pnd
import matplotlib.pyplot as plt

Spark imports

In [ ]:
import os
memory = '8g'
pyspark_submit_args = ' --driver-memory ' + memory + ' pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

In [ ]:
## spark imports
from pyspark.sql import Row, SparkSession
import pyspark.sql.functions as pyf

#spark = SparkSession.builder.master("local[1]").appName("RT0902").getOrCreate()
spark = SparkSession.builder.appName("RT0902").getOrCreate()

---
## Dataset
Les données originales viennent de Kaggle (https://www.kaggle.com/djonafegnem/chicago-crime-data-analysis)

On trouve une liste de crimes registrés par le département de police de Chicago.

Le dataset "réel" contient 4 fichiers pour des crimes allant de 2001 à 2017.
Comme le traitement de ces fichiers est long et demandeur en ressources, vous allez d'abord travailler avec un fichier réduit, qui ne contient que des données de 2001.

Une fois votre code "validé", vous pouvez l'utiliser sur le cloud pour traiter l'ensemble de fichiers de la police.

Ci-dessous vous trovez une description des différents champs des fichiers

In [ ]:
content_cols = '''
ID - Unique identifier for the record.
Case Number - The Chicago Police Department RD Number (Records Division Number), which is unique to the incident.
Date - Date when the incident occurred. this is sometimes a best estimate.
Block - The partially redacted address where the incident occurred, placing it on the same block as the actual address.
IUCR - The Illinois Unifrom Crime Reporting code. This is directly linked to the Primary Type and Description. See the list of IUCR codes at https://data.cityofchicago.org/d/c7ck-438e.
Primary Type - The primary description of the IUCR code.
Description - The secondary description of the IUCR code, a subcategory of the primary description.
Location Description - Description of the location where the incident occurred.
Arrest - Indicates whether an arrest was made.
Domestic - Indicates whether the incident was domestic-related as defined by the Illinois Domestic Violence Act.
Beat - Indicates the beat where the incident occurred. A beat is the smallest police geographic area – each beat has a dedicated police beat car. Three to five beats make up a police sector, and three sectors make up a police district. The Chicago Police Department has 22 police districts. See the beats at https://data.cityofchicago.org/d/aerh-rz74.
District - Indicates the police district where the incident occurred. See the districts at https://data.cityofchicago.org/d/fthy-xz3r.
Ward - The ward (City Council district) where the incident occurred. See the wards at https://data.cityofchicago.org/d/sp34-6z76.
Community Area - Indicates the community area where the incident occurred. Chicago has 77 community areas. See the community areas at https://data.cityofchicago.org/d/cauq-8yn6.
FBI Code - Indicates the crime classification as outlined in the FBI's National Incident-Based Reporting System (NIBRS). See the Chicago Police Department listing of these classifications at http://gis.chicagopolice.org/clearmap_crime_sums/crime_types.html.
X Coordinate - The x coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block.
Y Coordinate - The y coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block.
Year - Year the incident occurred.
Updated On - Date and time the record was last updated.
Latitude - The latitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block.
Longitude - The longitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block.
Location - The location where the incident occurred in a format that allows for creation of maps and other geographic operations on this data portal. This location is shifted from the actual location for partial redaction but falls on the same block.'''

### Données

Les données seront téléchargées et stockées dans `./data/`. Ce sont des fichiers .CSV.


In [ ]:
!mkdir data

In [ ]:
!gsutil -m cp -r gs://angelo_crime_data/*.csv ./data

In [ ]:
!ls -lh data/

---
## Lecture des données

Avec l'opération `csv read` de spark, nous allons lire et parser les fichiers. Le résultat sera un seul DataFrame :

In [ ]:
#df = spark.read.csv('gs://angelo_crime_data/Chicago_*.csv', inferSchema=True, header=True)
df = spark.read.csv('data/mini_data.csv', inferSchema=True, header=True)

Note : ce qui prend vraiment le temps est la découverte du schéma : on n'a pas tellement de lignes, après tout.

In [ ]:
# Affichage du schéma (structure) du dataframe
df.printSchema()

### Différences entre Pandas et Spark
Pandas a des opérations telles que `info()` et `describe()`. Dans Spark, on n'a que `describe()`, qui n'est pas comparable à celle de Pandas : il affiche plutôt le type des données, un peu comme `printSchema()`.

In [ ]:
df.describe()

Certaines lignes de n'ont aucune valeur déclarée à la colonne `location_description` . C'est le moment de les supprimer.

Pour cela, nous allons filtrer les entrées vides ('') en utilisant la fonction **`Dataset.filter`**.

In [ ]:
df = df.filter(df['location_description'] != '')

Un petit aperçu du début du dataframe :

In [ ]:
df.show(n=3, truncate=False)

On a quand même plus de 560 mille entrées sur le petit fichier !! 😵

In [ ]:
print(df.count())

---
## Comprendre les données

### Types de Crime

On veut connaître combien de types de crime (catégories) existent dans le fichier.

In [ ]:
# crime types
crime_type_groups = df.groupBy('primary_type').count()

In [ ]:
crime_type_counts = crime_type_groups.orderBy('count', ascending=False)

Jusqu'à ici ça a été rapide : Spark fait une exécution *lazy*, i.e., il n'a fait qu'enregistrer les *transformations* à applier. Il attendra pour lancer l'exécution uniquement lorsqu'une *action* est demandée (par exemple, afficher le résultat).

Dans la ligne suivante on demande le nombre total de lignes, mais en fait il va appliquer les modifications, faire le filtrage, etc. Sur un grand dataset, ça peut prendre pas mal de temps (d'où l'intérêt de distribuer le travail entre plusieurs machines).



La commande suivante affiche les 20 types de crime les plus fréquents :

In [ ]:
crime_type_counts.show(truncate=False)

On peut faire un affichage plus propre (et d'autres opérations) en transformant ce dataframe en Pandas :


In [ ]:
counts_pddf = crime_type_counts.toPandas()

In [ ]:
counts_pddf.head(10)

Ce dataset Pandas peut être utilisé pour une petite visualisation :

In [ ]:
plt.rcParams["figure.figsize"] = [10, 6]

counts_pddf.sort_values('count').plot(kind='barh', x='primary_type', y='count')

### Convertir les dates

**Si vous avez été attentif**, vous avez remarqué que les colonnes avec des dates ont été lues comme du texte simple :

In [ ]:
df.describe()

---
En effet, le schéma montrait que le champ `date` était de type `string`, ce qui n'est pas très utile.

Nous allons convertir ces dates au format timestamp.

Nous allons changer ce format afin de le lire sous la forme '02/23/2006 09:06:22 PM' , c'est à dire **`'MM/dd/yyyy hh:mm:ss a'`** (format américain).

On va aussi rajouter une colonne `month` qui indique le premier jour du mois, sans l'heure.

In [ ]:
from pyspark.sql.functions import to_timestamp, hour, trunc
# d'abord, on convertit 'date' avec to_timestamp() et on enregistre cette valeur dans 'date_time'
df = df.withColumn('date_time', to_timestamp('date', 'MM/dd/yyyy hh:mm:ss a'))
# ensuite, on crée une colonne 'month' à partir de 'datetime')
df = df.withColumn('month', trunc('date_time', 'month')) #adding a month column to be able to view stats on a monthly basis

In [ ]:
df.select(['date','date_time', 'month'])\
  .show(n=20, truncate=False)

### Combien d'arrestations ?

Pas tous les crimes donnent lieu à des arrestations.
Grâce à `groupBy`, nous allons afficher le nombre d'arrestations par mois (et le nombre de crimes sans arrestations).

In [ ]:
# On peut utiliser la colonne month pour affiche les quantités d'arrestations au fil des années, groupées par mois :
type_arrest_date = df.groupBy(['arrest', 'month'])\
                     .count()\
                     .orderBy(['month', 'count'], ascending=[True, False])
print()
type_arrest_date.show(truncate=False)

### Comment le nombre d'arrestations a évolué sur l'année ?

Pour l'afichage, nous allons encore une fois transformer notre dataset en Pandas. On transforme `type_arrest_date`, puis on utilise matplotlib pour l'affichage.

In [ ]:
# prompt: à partir de type_arrest_date, générer un graphique matplotlib affichant le nombre arrests true et false, par mois

import matplotlib.pyplot as plt

# Convert the Spark DataFrame to a Pandas DataFrame for easier plotting
type_arrest_date_pandas = type_arrest_date.toPandas()

# Create the plot
plt.figure(figsize=(10, 6))
for arrest_status in type_arrest_date_pandas['arrest'].unique():
    subset = type_arrest_date_pandas[type_arrest_date_pandas['arrest'] == arrest_status]
    plt.plot(subset['month'], subset['count'], label=f'Arrest: {arrest_status}')

plt.xlabel('Month')
plt.ylabel('Number of Arrests')
plt.title('Number of Arrests (True/False) per Month')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()


### À quel moment de la journée les criminels sont plus actifs ?

Ici c'est à vous de refaire le même type d'opération. Je vais juste vous montrer comment créer une colonne avec les heures.

In [ ]:
# Extract the "hour" field from the date into a separate column called "hour"
df_hour = df.withColumn('hour', hour(df['date_time']))

In [ ]:
# Derive a data frame with crime counts per hour of the day:
hourly_count = df_hour.groupBy(['primary_type', 'hour']).count()
hourly_total_count = hourly_count.groupBy('hour').sum('count')

In [ ]:
hourly_count_pddf = hourly_count.toPandas()
hourly_total_count_pddf = hourly_total_count.toPandas()

In [ ]:
hourly_count_pddf = hourly_count_pddf.sort_values(by='hour')
hourly_total_count_pddf = hourly_total_count_pddf.sort_values(by='hour')

In [ ]:
hourly_count_pddf.head(10)

In [ ]:
hourly_total_count_pddf.head(10)

In [ ]:
fig, ax = plt.subplots()
ax.plot(hourly_total_count_pddf['hour'], hourly_total_count_pddf['sum(count)'], label='Hourly Count')

ax.set(xlabel='Hour of Day', ylabel='Total records',
       title='Overall hourly crime numbers')
ax.legend()

Il semble que c'est plus agité entre 18h et 22h... avec un pic à midi.

Regardons le détail de chaque type de crime.



In [ ]:
import matplotlib.pyplot as plt

# Create the plot
plt.figure(figsize=(10, 6))

# Group data by hour and primary type and sum the counts
hourly_counts_grouped = hourly_count_pddf.groupby(['hour', 'primary_type'])['count'].sum().unstack()

# Plot stacked area chart
hourly_counts_grouped.plot(kind='area', stacked=True, ax=plt.gca())

plt.xlabel('Hour')
plt.ylabel('Number of Crimes')
plt.title('Hourly Crime Counts by Primary Type')
plt.xticks(rotation=45)
plt.legend(title='Primary Type', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


### Dans que type d'endroit les crimes sont commis ?

Le dataset indique la "classe" de lieu où le crime a été commis : maison, rue, etc. Regardons ça en détails.

In [ ]:
# Combien de types d'endroit sont recensés
df.select('location_description').distinct().count()

Ok, il y a 114 types différents d'endroit qui sont recensés.

Quels sont les 10 endroits les plus fréquents ?

In [ ]:
df.groupBy(['location_description']).count().orderBy('count', ascending=False).show(10, truncate=False)

### Crimes "domestiques" :

En dehors de la localité, le dataset indique aussi s'il s'agit d'une violence domestique (dispute familiale, harcélement, etc.) ou pas.

Regardons ces types de violence plus en détails :

In [ ]:
domestic_hour = df_hour.groupBy(['domestic', 'hour']).count().orderBy('hour').toPandas()

In [ ]:
import matplotlib.pyplot as plt

# Filter for domestic cases
domestic_cases = df_hour.filter(df_hour['domestic'] == True)

# Group by hour and count
domestic_cases_by_hour = domestic_cases.groupBy('hour').count().orderBy('hour').toPandas()

# Create the bar plot
#plt.figure(figsize=(8, 4))
domestic_cases_by_hour.plot(kind='bar', x='hour', y='count', ax=plt.gca())
plt.xlabel('Hour of the Day')
plt.ylabel('Number of Domestic Cases')
plt.title('Number of Domestic Cases per Hour')
plt.xticks(rotation=45)
plt.show()


Il y a une montée des violences domestiques le soir, avec un pic isolé à midi (disputes pendant le repas ?)

Et comment ça se situe par rapport aux crimes/violences "non-domestiques" ?

In [ ]:
import matplotlib.pyplot as plt
# Create the plot
plt.figure(figsize=(10, 4))

# Group data by hour and domestic status and sum the counts
domestic_counts_grouped = domestic_hour.groupby(['hour', 'domestic'])['count'].sum().unstack()

# Plot stacked bar chart
domestic_counts_grouped.plot(kind='bar', stacked=True, ax=plt.gca())

plt.xlabel('Hour')
plt.ylabel('Number of Crimes')
plt.title('Hourly Crime Counts by Domestic Status')
plt.xticks(rotation=45)
plt.legend(title='Domestic', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


### Une analyse par rapport au temps

Les données de type heure/date permettent d'obtenir plus d'information sur les types de crime et d'émettre des hypothèses sur leurs sursauts. Par ontre, d'autres facteurs externes comme le changement de garde ou les nouvelles politiques de sécurité peuvent avoir un impact non décrit ici.

Néanmoins, si on a une idée de quand et où les crimes sont les plus fréquents, on peut s'aventurer à faire quelques prévisions...

On va rajouter quelques champs utiles :

 * l'heure du jour (déjà présent dans le champ 'hour')
 * le jour de la semaine (dimanche = 1, ..., samedi = 7)
 * le mois de l'année
 * le "numéro du jour" dans une séquence 1, 2...(on commence à compter à partir du 2001-01-01).

In [ ]:
from pyspark.sql.functions import dayofweek, month, dayofmonth, datediff, to_date, lit

df_dates = df_hour.withColumn('week_day', dayofweek(df_hour['date_time']))\
                 .withColumn('year_month', month(df_hour['date_time']))\
                 .withColumn('month_day', dayofmonth(df_hour['date_time']))\
                 .withColumn('date_number', datediff(df['date_time'], to_date(lit('2001-01-01'), format='yyyy-MM-dd')))\
                 .cache()

In [ ]:
df_dates.select(['date', 'month', 'hour', 'week_day', 'year', 'year_month', 'month_day', 'date_number']).show(10, truncate=False)

## Les crimes par rapport au jour de la semaine


In [ ]:
week_day_crime_counts = df_dates.groupBy('week_day').count()

In [ ]:
week_day_crime_counts_pddf = week_day_crime_counts.orderBy('week_day').toPandas()

In [ ]:
week_day_crime_counts_pddf.plot(kind='bar', x='week_day', y='count')

On voit très peu de variance... D'un autre côté, les criminels restent "méchants" tous les jours. Et probablemnt il y a des crimes le dimanche qui ne sont signalés que le lundi !

## Mois de l'année



In [ ]:
year_month_crime_counts = df_dates.groupBy('year_month').count()

In [ ]:
year_month_crime_counts_pddf = year_month_crime_counts.orderBy('year_month').toPandas()

Il semble que la période Mai-Août est la plus active pour les criminels. Des idées sur la cause ?


In [ ]:
year_month_crime_counts_pddf.plot(y='count', x='year_month', kind='bar')

AH, ça c'est intéressant ! On a beaucoup de crimes en janvier et février. Serait-ça lié à la déprim de l'hiver ? Regardons rapidement si ça a un impact sur les violences domestiques.

In [ ]:
domestic_month = df_dates.groupBy('domestic','year_month').count().orderBy('year_month').toPandas()

In [ ]:
domestic_month = domestic_month[domestic_month['domestic'] == True]

In [ ]:
year_month_crime_counts_pddf['domestic_count']=domestic_month['count']
year_month_crime_counts_pddf['domestic_percent'] = domestic_month['count']/year_month_crime_counts_pddf['count']

In [ ]:
year_month_crime_counts_pddf.plot(x='year_month', y='domestic_percent', kind='bar', color='orange')

Bien que les mois d'hiver présentent un taux élevé de violences domestiques, le "blues de l'hiver" ne semble pas avoir une influence si grande que ça. 😯

---


# Pouvons-nous prédire la catégorie de crime (`primary_type`) à partir de quelques caractéristiques (domestique, avec violence), l'endroit (district, community_area), etc. ?

Afin de le faire, on va nettoyer un peu plus le dataset.

Tout d'abord, essayons de supprimer quelques colonnes qui ne sont pas intéressantes ou qui risquent d'influencer trop le dataset :

 * 'id'
 * 'case_number'
 * 'date' - déjà présent dans les autres données de date/heure
 * 'block' - trop précis, peut ajouter du "bruit"
 * 'iucr' - c'est juste un code pour le type de crime
 * 'x_coordinate' - trop précis, peut ajouter du "bruit"
 * 'y_coordinate' - trop précis, peut ajouter du "bruit"
 * 'year' - déjà présent dans les autres données de date/heure
 * 'updated_on' - pas utile
 * 'latitude' - trop précis, peut ajouter du "bruit"
 * 'longitude' - trop précis, peut ajouter du "bruit"
 * 'location' - non inclus
 * 'date_time' - trop précis, peut ajouter du "bruit"
 * 'description' - Supprimé. On trouvera l'équivalent dans `primary type`, qui est **notre objectif**

 On pourrait faire ça avec des `drop()`, mais faisons différemment : avec Spark nous avons la fonction `select()`, donc essayons de passer plutôt la liste de features qu'on veut garder :

 * 'location_description'
 * 'arrest'
 * 'domestic'
 * 'beat'
 * 'district'
 * 'ward'
 * 'community_area'
 * 'fbi_code'
 * 'hour'
 * 'week_day'
 * 'year_month'
 * 'month_day'
 * 'date_number'
 * 'primary_type'


In [ ]:
selected_features = [
 'location_description',
 'arrest',
 'domestic',
 'beat',
 'district',
 'ward',
 'community_area',
 'fbi_code',
 'hour',
 'week_day',
 'year_month',
 'month_day',
 'date_number',
 'primary_type']

In [ ]:
features_df = df_dates.select(selected_features)

Nous allons aussi identifier les types "uniques" pour les différents types de features (quels types de "location_description", quels types de "arrest"...). Ça sera utile pour la conversion des données catégoriques.

In [ ]:
feature_level_count_dic = []

for feature in selected_features:
    print('Analysing %s' % feature)
    levels_list_df = features_df.select(feature).distinct()
    feature_level_count_dic.append({'feature': feature, 'level_count': levels_list_df.count()})


In [ ]:
pnd.DataFrame(feature_level_count_dic).sort_values(by='level_count', ascending=False)

### Preparer le modèle

On fait un premier passage pour supprimer les "cases vides" :

In [ ]:
df_dates_features = features_df.dropna(subset=selected_features)

In [ ]:
df_dates_features.show(5)

In [ ]:
df_dates_features.printSchema()

Les features retenues sont des **catégories**, donc nous devons passer par un encodeur pour les transformer en valeurs numériques.

Sur ScikitLearn on pourrait utiliser `OrdinalEncoder`ou `OneHotEncoder`, mais ça risque de ne pas fonctionner si on a beaucoup de données.

Spark offre ses propres versions d'encodeurs. Ici, nous voulons utiliser `StringIndexer`, un encodeur qui fonctionne comme OrdinalEncoder de sklearn.

Attention : StringIndexer ne reconnaît que les colonnes de format String (🙃). Il faudra transformer les colonnes *arrest* et *domestic* en string, car pour le moment elles sont de type booléen !

In [ ]:
df_dates_features = df_dates_features.withColumn('arrest', df_dates_features['arrest'].cast('string'))
df_dates_features = df_dates_features.withColumn('domestic', df_dates_features['domestic'].cast('string'))

Là ça doit être bon !

Utilisons le string indexer de Spark pour transformer les catégories des features séléctionnées.

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler


In [ ]:
for feature in feature_level_count_dic:
    indexer = StringIndexer(inputCol=feature['feature'], outputCol='%s_indexed' % feature['feature'])
    print('Fitting feature "%s"' % feature['feature'])
    model = indexer.fit(df_dates_features)
    print('Transforming "%s"' % feature['feature'])
    df_dates_features = model.transform(df_dates_features)

Comme on peut voir, on vient de créer plusieurs colonnes suppélentaires (suffixe _indexed) qui contiennent des valeurs numériques.

In [ ]:
df_dates_features.show(5)


Maintenant, on va vectoriser les éléments pour les avoir dans une colonne `features`. En effet, Spark n'utilise pas des Dataframe directement mais a besoin qu'on transforme les données en vecteurs.

In [ ]:
indexed_features = ['%s_indexed' % fc['feature'] for fc in feature_level_count_dic]
indexed_features

In [ ]:
assembler = VectorAssembler(inputCols=indexed_features, outputCol='features')
vectorized_df_dates = assembler.transform(df_dates_features)

In [ ]:
vectorized_df_dates.select('features').take(1)

### Et entraîner le modèle.

Utiliser une répartition **60%**/**40%** entre les données train et test.

Pour commencer, utilisons une régression logistique.
On peut voir l'ensemble de méthodes supportées par [Spark ici.](https://spark.apache.org/docs/latest/ml-classification-regression.html)

In [ ]:
train, test = vectorized_df_dates.randomSplit([0.6, 0.4])

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
logisticRegression = LogisticRegression(labelCol='primary_type_indexed', featuresCol='features', maxIter=10, family='multinomial')

In [ ]:
fittedModel = logisticRegression.fit(train)

## Quelle est la performance du modèle ?

In [ ]:
fittedModel.summary.accuracy

#### Est-ce que ça semble un bon modèle pour prédire les crimes ?

## À vous:

 * Exécuter le modèle sur l'ensemble de test

In [ ]:
predictions = fittedModel.transform(test)
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="primary_type_indexed", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy on test data = {accuracy}")

On a environ 83% d'accuracy. Ceci dit, sur 30 catégories de `primary_index`, il y a certaines qui doivent être très bien prédites et d'autres pas.